In [1]:
from utils import config, data
from utils.seq2seq.batcher import *
from utils.seq2seq.train_util import *
from utils.seq2seq.rl_util import *
from utils.seq2seq.initialize import loadCheckpoint, save_model
from utils.seq2seq.write_result import *
from datetime import datetime as dt
from tqdm import tqdm
from translate.seq2seq_beam import *
from tensorboardX import SummaryWriter
import argparse
from utils.seq2seq.rl_util import *
from torch.distributions import Categorical

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

parser = argparse.ArgumentParser()
parser.add_argument('--key_attention', type=bool, default=False, help = 'True/False')
parser.add_argument('--intra_encoder', type=bool, default=False, help = 'True/False')
parser.add_argument('--intra_decoder', type=bool, default=False, help = 'True/False')
parser.add_argument('--copy', type=bool, default=True, help = 'True/False') # for transformer

parser.add_argument('--model_type', type=str, default='seq2seq', choices=['seq2seq', 'transformer'])
parser.add_argument('--train_rl', type=bool, default=False, help = 'True/False')
parser.add_argument('--keywords', type=str, default='POS_keys', 
                    help = 'POS_keys / DEP_keys / Noun_adj_keys / TextRank_keys')

parser.add_argument('--lr', type=float, default=0.0001)
parser.add_argument('--rand_unif_init_mag', type=float, default=0.02)
parser.add_argument('--trunc_norm_init_std', type=float, default=0.001)
parser.add_argument('--mle_weight', type=float, default=1.0)
parser.add_argument('--gound_truth_prob', type=float, default=0.5)

parser.add_argument('--max_enc_steps', type=int, default=500)
parser.add_argument('--max_dec_steps', type=int, default=20)
parser.add_argument('--min_dec_steps', type=int, default=6)
parser.add_argument('--max_epochs', type=int, default=15)
parser.add_argument('--vocab_size', type=int, default=50000)
parser.add_argument('--beam_size', type=int, default=5)
parser.add_argument('--batch_size', type=int, default=8)

parser.add_argument('--hidden_dim', type=int, default=512)
parser.add_argument('--emb_dim', type=int, default=300)
parser.add_argument('--gradient_accum', type=int, default=1)

parser.add_argument('--load_ckpt', type=str, default='0760000', help='0002000')
parser.add_argument('--word_emb_type', type=str, default='word2Vec', help='word2Vec/glove/FastText')
parser.add_argument('--pre_train_emb', type=bool, default=True, help = 'True/False') # 若pre_train_emb為false, 則emb type為NoPretrain


opt = parser.parse_args(args=[])
config = re_config(opt)
loggerName, writerPath = getName(config)    
logger = getLogger(loggerName)
writer = SummaryWriter(writerPath)

I0424 10:16:59.053524 139653603854144 file_utils.py:35] PyTorch version 1.4.0 available.
2020-04-24 10:16:59 - Pointer_generator_word2Vec - INFO: - logger已啟動
I0424 10:16:59.892058 139653603854144 train_util.py:102] logger已啟動


In [2]:
total_df = getDataLoader(logger, config)[-1]

2020-04-24 10:17:16 - Pointer_generator_word2Vec - INFO: - train : 119524, test : 13281
I0424 10:17:16.826524 139653603854144 batcher.py:174] train : 119524, test : 13281
2020-04-24 10:17:16 - Pointer_generator_word2Vec - INFO: - train batches : 14940, test batches : 1660
I0424 10:17:16.886122 139653603854144 batcher.py:185] train batches : 14940, test batches : 1660


In [3]:
with tqdm(total=len(total_df)) as pbar:
    for i ,row in total_df.iterrows():        
        pbar.update(1)
        pbar.set_description("row %s " % (i))

        art_set = row['review'].strip().split(' ')        
        ref_set = row['summary'].strip().split(' ')
        overlap = set(art_set) & set(ref_set)
        total_df.loc[i, 'overlap'] = len(overlap)


total_df.head()
total_df.describe()

row 118171 :  89%|████████▉ | 118172/132805 [05:18<00:33, 436.05it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

row 129943 :  98%|█████████▊| 129944/132805 [05:49<00:08, 355.85it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
total_df = total_df.sort_values(by=['lemm_review_len','overlap'], ascending = False)
total_df.head()

,review,summary,cheat,cheat_num,lemm_review_len,lemm_summary_len,POS_keys,DEP_keys,TextRank_keywords,overlap
22919,. although should thank amazon warehouse for f...,<s> after year of drag foot this tv make me ju...,"{'tv', 'year', 'hd'}",3,1000,14,"pair mine,class nbsp,speed high,class nbsp,ite...","avsfourm great,inventory list,avsfourm great,i...",use nbsp light mount mounting regard picture u...,7.0
100978,div class div input type hide name value class...,<s> nice look and highly portable battery that...,"{'nice', 'battery', 'efficiency'}",3,1000,14,"phone flip,unit video,cable proper,button pres...","phone small,phone flip,cable proper,phone smar...",charging device charge battery pack testing ma...,7.0
29057,note please see the comment section for the li...,<s> the new tamron mm macro vc is winner side ...,"{'winner', 'tamron', 'macro', 'mm'}",4,999,14,"customer reveal,review discuss,lense macro,qua...","offer model,offer lense,nikon canon,sony canon...",image canon focus mm macro lense lens tamron c...,6.0
24114,handsome camera with vintage vibe the harken b...,<s> vintage vibe precision build great image </s>,"{'vibe', 'great', 'vintage', 'image'}",4,999,8,"camera handsome,vibe vintage,grip thin,pack co...","camera handsome,vibe vintage,grip thin,grip la...",camera olympus use lcd iso dial evf evfs batte...,4.0
80518,after much research buy the concept2 model row...,<s> concept model has advantage over bodycraft...,"{'model', 'advantage', 'bodycraft'}",3,999,9,"follow compare,ymca local,wind 2nd,monitor pol...","ymca local,store 2nd,software rowpro,concept2 ...",concept online row rowing market marketing row...,4.0


In [10]:
train_df, val_df = train_test_split(total_df, test_size=0.1, 
                                    random_state=0, shuffle=True)

In [17]:
len(train_df[train_df['overlap']>5])/len(train_df)

0.270774070479569

In [18]:
len(val_df[val_df['overlap']>5])/len(val_df)

0.2754310669377306